In [12]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from keras.applications.efficientnet import preprocess_input

import os
import cv2
from os import walk
import glob as gb
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [13]:
SEED = 1000
IMG_SIZE = 224
BATCH_SIZE = 32

TRAIN_DIR = '../input/caltech-zipped/Caltech Bird 2/Train'
VALID_DIR = '../input/caltech-zipped/Caltech Bird 2/Validation'
TEST_DIR = '../input/caltech-zipped/Caltech Bird 2/Test'

In [14]:
from glob import glob
training_dir=TRAIN_DIR
validation_dir=VALID_DIR
image_files = glob(training_dir + '/*/*')
valid_image_files = glob(validation_dir + '/*/*')

In [15]:
folders = glob(training_dir + '/*')
num_classes = len(folders)
print ('Total Classes = ' + str(num_classes))

Total Classes = 199


In [16]:
from keras.models import Model
from keras.layers import Flatten, Dense
from tensorflow.keras.applications import EfficientNetB0

#IMG_SIZE=80

IMAGE_SIZE = [IMG_SIZE, IMG_SIZE] 


En = EfficientNetB0(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False) 


for layer in En.layers:
    layer.trainable = False

x = Flatten()(En.output)

x = Dense(num_classes, activation = 'softmax')(x)  

model = Model(inputs = En.input, outputs = x)



In [17]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling_1 (Rescaling)         (None, 224, 224, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
normalization_1 (Normalization) (None, 224, 224, 3)  7           rescaling_1[0][0]                
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           normalization_1[0][0]            
____________________________________________________________________________________________

In [18]:
training_datagen = ImageDataGenerator(
                                    rescale=1./255,
                                    shear_range=0.2, 
                                    zoom_range=0.2,
                                    horizontal_flip=False,
                                    preprocessing_function=preprocess_input)

validation_datagen = ImageDataGenerator(rescale = 1./255, preprocessing_function=preprocess_input)

training_generator = training_datagen.flow_from_directory(
    training_dir, 
    target_size = IMAGE_SIZE, 
    batch_size = BATCH_SIZE, 
    class_mode = 'sparse')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir, 
    target_size = IMAGE_SIZE, 
    batch_size = BATCH_SIZE, 
    class_mode = 'sparse')

Found 8190 images belonging to 199 classes.
Found 2311 images belonging to 199 classes.


In [19]:
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 40:
        lr *= 0.5e-3
    elif epoch > 30:
        lr *= 1e-3
    elif epoch > 20:
        lr *= 1e-2
    elif epoch > 10:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr
lr_scheduler = LearningRateScheduler(lr_schedule)

In [20]:
model.compile(
    optimizer=keras.optimizers.Adam(lr=lr_schedule(0)), 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

Learning rate:  0.001


In [21]:
h = model.fit(training_generator, 
              epochs=50,
              steps_per_epoch = 8190 // BATCH_SIZE,
              validation_data = validation_generator,
              validation_steps = 2311 // BATCH_SIZE,
              callbacks=lr_scheduler,
              verbose=2
             )

Epoch 1/50
Learning rate:  0.001
255/255 - 167s - loss: 34.8595 - accuracy: 0.0066 - val_loss: 19.1179 - val_accuracy: 0.0048
Epoch 2/50
Learning rate:  0.001
255/255 - 126s - loss: 21.0369 - accuracy: 0.0044 - val_loss: 20.5771 - val_accuracy: 0.0043
Epoch 3/50
Learning rate:  0.001
255/255 - 127s - loss: 20.9931 - accuracy: 0.0050 - val_loss: 21.9647 - val_accuracy: 0.0052
Epoch 4/50
Learning rate:  0.001
255/255 - 127s - loss: 23.3336 - accuracy: 0.0037 - val_loss: 21.8625 - val_accuracy: 0.0052
Epoch 5/50
Learning rate:  0.001
255/255 - 127s - loss: 23.6998 - accuracy: 0.0048 - val_loss: 23.9287 - val_accuracy: 0.0052
Epoch 6/50
Learning rate:  0.001
255/255 - 126s - loss: 25.1240 - accuracy: 0.0042 - val_loss: 22.9985 - val_accuracy: 0.0052
Epoch 7/50
Learning rate:  0.001
255/255 - 127s - loss: 25.6174 - accuracy: 0.0061 - val_loss: 28.1347 - val_accuracy: 0.0052
Epoch 8/50
Learning rate:  0.001
255/255 - 126s - loss: 26.2148 - accuracy: 0.0053 - val_loss: 25.0928 - val_accuracy:

from glob import glob
training_dir=TRAIN_DIR
validation_dir=VALID_DIR
image_files = glob(training_dir + '/*/*')
valid_image_files = glob(validation_dir + '/*/*')